<a href="https://colab.research.google.com/github/tinachentc/kernel-learning-in-ridge-regression/blob/master/kernel_learning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kernel Learning in Ridge Regression Example

The following code demonstrates low-rankness of the solution matrix $\Sigma$ in a variant of kernel ridge regression.

<!-- $$\mathop{\rm minimize}_{f, \gamma, \Sigma} ~~~\frac{1}{2} \mathbb{E}_n \left[(Y - f(X) - \gamma)^2\right] + \frac{\lambda}{2} \Vert f\Vert _{\mathcal{H}_\Sigma}^2 ~~~ \mathop{\rm subject\ to}  ~~~~\Sigma \succeq 0. $$
This RKHS, denoted as $\mathcal{H}_\Sigma$, utilizes a kernel
$$(x,x') \mapsto k_\Sigma(x, x')  = \phi(\Vert x-x'\Vert^2_\Sigma)$$
where $\Vert x-x'\Vert_\Sigma = \sqrt{(x-x')^T \Sigma (x-x')}$, and $\phi$ is a real-valued function so that $k_\Sigma$ is a kernel for every positive semidefinite $\Sigma$.  -->
 <!-- where $\phi(z) = \exp(-z)$ -->
Gaussian kernel is used in this example. For simulation setting, the feature vector $X\sim N(0,I)$, and the response $Y=f(X)+\epsilon$, where $f(x)=x_1 x_2$ and $\epsilon\sim N(0,\sigma^2)$.

For more details, please refer to our paper:
[Kernel Learning in Ridge Regression “Automatically” Yields Exact Low Rank Solution](https://arxiv.org/abs/2310.11736).


In [23]:
import torch
import numpy as np
from scipy.linalg import eigh
from numpy.linalg import solve, inv, norm
from tqdm import tqdm
from sklearn.metrics import accuracy_score

import pandas as pd
import argparse

## Set Arguments

In [34]:
# set simulation arguments
seed = 1 #seed
n = 500 #sample size
d = 30 #sample dimension
e = 0.1 #noise size
rho = 0. #sample correlation

# set model arguments
lam = 0.1 #ridge regularization parameter

# set optimization arguments
iters = 2000 #maximum number of epoches
lr = 0.1 #initial learning rate
alpha = 0.001 #Armijo parameter
beta = 0.5 #Armijo parameter
tol = 0.001 #tolerance

In [30]:
def euclidean_distances_M(samples, centers, M, squared=True):

    samples_norm = (samples @ M)  * samples
    samples_norm = torch.sum(samples_norm, dim=1, keepdim=True)

    if samples is centers:
        centers_norm = samples_norm
    else:
        centers_norm = (centers @ M) * centers
        centers_norm = torch.sum(centers_norm, dim=1, keepdim=True)

    centers_norm = torch.reshape(centers_norm, (1, -1))

    distances = samples.mm(M @ torch.t(centers))
    distances.mul_(-2)
    distances.add_(samples_norm)
    distances.add_(centers_norm)
    if not squared:
        distances.clamp_(min=0)
        distances.sqrt_()

    return distances

def gaussian_M(samples, centers, bandwidth, M):
    assert bandwidth > 0
    kernel_mat = euclidean_distances_M(samples, centers, M)
    kernel_mat.clamp_(min=0)
    gamma = 1. / bandwidth
    kernel_mat.mul_(-gamma)
    kernel_mat.exp_()
    return kernel_mat

In [26]:
def get_grads(reg, X, Y, L, P, batch_size, unfold=0):
    d, _ = P.shape
    n, c = Y.shape
    batches_X = torch.split(X, batch_size)
    batches_Y = torch.split(Y, batch_size)
    G = torch.zeros(d, d)
    for i in tqdm(range(len(batches_X))):
        x = batches_X[i]
        y = batches_Y[i]
        m, _ = x.shape
        K = gaussian_M(x, x, L, P)

        # calculate alpha and diff
        proj = torch.eye(m)-torch.ones((m,m))/m
        a = solve(proj@K + reg * np.eye(m), proj@y)
        a = torch.from_numpy(a).float()
        diff = x.unsqueeze(1) - x.unsqueeze(0)

        # separate unfold cases
        if unfold == 0:
            res = torch.einsum('ijk,ijl->ijkl', diff, diff)
            C = torch.mul(res, K.view(m,m,1,1))
            del res
            G0 = torch.tensordot(a, C, dims=([0], [0]))
            del C
            G += torch.sum(torch.tensordot(torch.sum(G0, dim=0), a, dims=([0], [0])), dim=2)
            del G0

        elif unfold == 1:
            for j in range(d):
                resj = torch.einsum('ijk,ijl->ijkl', diff, diff[:, :, j:(j + 1)]).squeeze()
                Cj = torch.mul(resj, K.view(m,m,1))
                del resj
                Gj = torch.tensordot(a, Cj, dims=([0], [0]))
                del Cj
                G[j] += torch.sum(torch.tensordot(torch.sum(Gj, dim=0), a, dims=([0], [0])), dim=1)
                del Gj

        else:
            for j in range(d):
                for k in range(d):
                    resjk = torch.einsum('ijk,ijl->ijkl', diff[:, :, j:(j + 1)], diff[:, :, k:(k + 1)]).squeeze()
                    Cjk = torch.mul(resjk, K)
                    del resjk
                    Gjk = torch.tensordot(a, Cjk, dims=([0], [0]))
                    G[j,k] += torch.sum(torch.tensordot(torch.sum(Gjk, dim=0), a, dims=([0], [0])), dim=0)
                    del Gjk

    G *= reg/n/L
    G = G.numpy()
    return G

def trainSig(X_train, y_train, X_test, y_test, M, d0,
        iters, batch_size, reg, lr0,
        alpha, beta, tol, X_val=None, y_val=None, classification=0, unfold=0):
    L = 1
    n, d = X_train.shape
    mm, d = X_test.shape
    if X_val is not None and y_val is not None:
        m, d = X_val.shape

    U = np.eye(d, dtype='float32')
    obj_seq = np.zeros(iters+1)
    err_seq = np.zeros(iters+1); terr_seq = np.zeros(iters+1)
    rank_seq = np.zeros(iters+1)

    proj = torch.eye(n) - torch.ones((n, n)) / n


    i = 0
    # rank
    rank_seq[i] = d0

    # training err
    K_train = gaussian_M(X_train, X_train, L, torch.from_numpy(M)).numpy()
    sol = solve(proj @ K_train + reg * np.eye(n), proj @ y_train).T
    preds0 = (sol @ K_train).T
    csol = np.mean(y_train.numpy() - preds0)
    preds = preds0 + np.ones((n, 1)) * csol
    terr_seq[i] = np.sqrt(np.mean(np.square(preds - y_train.numpy())))
    print("Round " + str(i) + " Train RMSE: ", terr_seq[i])

    # validation err
    if X_val is not None and y_val is not None:
        K_val = gaussian_M(X_train, X_val, L, torch.from_numpy(M)).numpy()
        preds = (sol @ K_val).T + np.ones((m, 1)) * csol
        if classification:
            err_seq[i] = 1. - accuracy_score(preds > 0.5, y_val.numpy())
            print("Round " + str(i) + " Val Acc: ", 1. - err_seq[i])
        else:
            err_seq[i] = np.sqrt(np.mean(np.square(preds - y_val.numpy())))
            print("Round " + str(i) + " Val RMSE: ", err_seq[i])

    # objective function value
    obj = sol @ (K_train + reg * np.eye(n)) @ sol.T
    obj_seq[i] = obj[0, 0] * reg / n / 2
    print("Round " + str(i) + " Obj: ", obj_seq[i])


    for i in range(iters):
        # shuffle and calculate gradient
        indices = torch.randperm(X_train.shape[0])
        G = get_grads(reg, X_train[indices], y_train[indices], L, torch.from_numpy(M), batch_size=batch_size, unfold=unfold).astype('float32')

        # PGD with Armijo rule
        lr = lr0
        M0 = M - lr * G
        D, V = eigh(M0)
        if sum(D < 0) > 0:
            D[D < 0] = 0
            M0 = V@np.diag(D)@inv(V)
        K_train_upd = gaussian_M(X_train, X_train, L, torch.from_numpy(M0)).numpy()
        sol_upd = solve(proj@K_train_upd + reg * np.eye(n), proj@y_train).T
        obj_upd = sol_upd @ (K_train_upd + reg * np.eye(n)) @ sol_upd.T
        f_upd = obj_upd[0,0]*reg/n/2
        Gp = U@U.T@G@U@U.T
        while f_upd > obj_seq[i] - alpha * lr * np.trace(Gp.T@Gp):
           lr *= beta
           M0 = M - lr * G
           D, V = eigh(M0)
           if sum(D < 0) > 0:
               D[D < 0] = 0
               M0 = V @ np.diag(D) @ inv(V)
           K_train_upd = gaussian_M(X_train, X_train, L, torch.from_numpy(M0)).numpy()
           sol_upd = solve(proj@K_train_upd + reg * np.eye(n), proj@y_train).T
           obj_upd = sol_upd @ (K_train_upd + reg * np.eye(n)) @ sol_upd.T
           f_upd = obj_upd[0,0]*reg/n/2
           if lr < 1e-15:
               lr = 0.
               break
           print(lr)

        print(norm(M-M0, 'fro')/lr)
        if lr == 0 or norm(M-M0, 'fro')/lr < tol:
            obj_seq[i+1] = obj_seq[i]
            terr_seq[i+1] = terr_seq[i]
            err_seq[i+1] = err_seq[i]
            rank_seq[i+1] = rank_seq[i]
            break
        else:
            M = M0
        U = V@np.diag(np.sqrt(D))
        U = U[:,D>0]
        print(D)

        # rank
        rank_seq[i+1] = sum(D > 0)

        # training err
        K_train = gaussian_M(X_train, X_train, L, torch.from_numpy(M)).numpy()
        sol = solve(proj @ K_train + reg * np.eye(n), proj @ y_train).T
        preds0 = (sol @ K_train).T
        csol = np.mean(y_train.numpy() - preds0)
        preds = preds0 + np.ones((n, 1)) * csol
        terr_seq[i+1] = np.sqrt(np.mean(np.square(preds - y_train.numpy())))
        print("Round " + str(i+1) + " Train RMSE: ", terr_seq[i+1])

        # validation err
        if X_val is not None and y_val is not None:
            K_val = gaussian_M(X_train, X_val, L, torch.from_numpy(M)).numpy()
            preds = (sol @ K_val).T + np.ones((m, 1)) * csol
            if classification:
                err_seq[i+1] = 1. - accuracy_score(preds>0.5, y_val.numpy())
                print("Round " + str(i+1) + " Val Acc: ", 1. - err_seq[i+1])
            else:
                err_seq[i+1] = np.sqrt(np.mean(np.square(preds - y_val.numpy())))
                print("Round " + str(i+1) + " Val RMSE: ", err_seq[i+1])

        # objective function value
        obj = sol @ (K_train + reg * np.eye(n)) @ sol.T
        obj_seq[i+1] = obj[0, 0] * reg / n / 2
        print("Round " + str(i+1) + " Obj: ", obj_seq[i+1])

    # test err
    K_test = gaussian_M(X_train, X_test, L, torch.from_numpy(M)).numpy()
    preds = (sol @ K_test).T + np.ones((mm, 1)) * csol
    if classification:
        test_err = 1. - accuracy_score(preds > 0.5, y_test.numpy())
        print("Test Acc: ", 1. - test_err)
    else:
        test_err = np.sqrt(np.mean(np.square(preds - y_test.numpy())))
        print("Test RMSE: ", test_err)

    return test_err, M, obj_seq[0:i+2], terr_seq[0:i+2], err_seq[0:i+2], rank_seq[0:i+2]


## Generate Simulation

In [35]:
# generate noise
np.random.seed(seed)
u = e*np.random.normal(0, 1, size=n)

# generate x
np.random.seed(seed * 200)
s = np.zeros((d, d))
for i in range(d):
  for j in range(d):
    s[i][j] = rho ** abs(i - j)
X = np.random.multivariate_normal(np.zeros(d), s, size=n)

# generate y
y0 = X[:, 0] * X[:, 1]
y = y0 + u

X = torch.from_numpy(X).float()
y = torch.from_numpy(y.reshape(n, 1)).float()

## Run Model

In [36]:
terr, M, obj_seq, tval_seq, val_seq, rank_seq = trainSig(X_train=X, y_train=y, X_test=X, y_test=y,
                                iters=iters, batch_size=n, reg=n*lam, lr0=lr,
                                M=np.eye(d, dtype='float32')/d, d0=d, alpha=alpha, beta=beta, tol=tol)


Round 0 Train RMSE:  0.8780049746562272
Round 0 Obj:  0.39385953323674244


100%|██████████| 1/1 [00:02<00:00,  2.24s/it]


0.1783749833703041
[0.03059593 0.03083538 0.03153603 0.03159735 0.03165667 0.03194415
 0.03206563 0.03213494 0.03219572 0.03228062 0.03243659 0.03249811
 0.03269414 0.03279128 0.03287331 0.03288353 0.03291719 0.03306768
 0.03321145 0.03325896 0.03329459 0.03339814 0.03345781 0.03369589
 0.03385678 0.0339073  0.03399613 0.03976917 0.04184272 0.04637579]
Round 1 Train RMSE:  0.8745702359100312
Round 1 Obj:  0.3922643104033795


100%|██████████| 1/1 [00:02<00:00,  2.59s/it]


0.18420526757836342
[0.02660732 0.02730062 0.02882151 0.02909785 0.02937746 0.02979062
 0.03025682 0.03039553 0.03050077 0.03064344 0.03100485 0.03119126
 0.03168678 0.03188758 0.03196459 0.03205732 0.03212794 0.03244208
 0.0327179  0.03293505 0.03303897 0.03319486 0.03337091 0.03375829
 0.03380999 0.03405211 0.03446184 0.04602565 0.05134518 0.05796076]
Round 2 Train RMSE:  0.870720609641905
Round 2 Obj:  0.39048100184689555


...

0.0010228242899756879
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.44411856 0.5292237 ]
Round 218 Train RMSE:  0.6834810579449794
Round 218 Obj:  0.3005733174635515


100%|██████████| 1/1 [00:01<00:00,  1.71s/it]


0.001007503960863687
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.44415054 0.5293175 ]
Round 219 Train RMSE:  0.6834819739558657
Round 219 Obj:  0.30057326677689067


100%|██████████| 1/1 [00:01<00:00,  1.79s/it]

0.0009903889440465719
Test RMSE:  0.6834819739558657
